# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0027, -0.1434, -0.1546, -0.0132, -0.0526, -0.0805,  0.0701, -0.0266,
         -0.3559,  0.0209],
        [-0.0266, -0.0299, -0.1644,  0.0258,  0.0119, -0.0249,  0.0197, -0.1271,
         -0.2947, -0.0362]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0485,  0.0470,  0.0624, -0.0909,  0.1269,  0.1685,  0.1925,  0.0550,
          0.0284, -0.0801],
        [ 0.0298, -0.0346,  0.0377, -0.0936, -0.0451,  0.1722,  0.1295,  0.0615,
          0.0682, -0.0011]], grad_fn=<AddmmBackward0>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0077,  0.0073, -0.1978,  0.0472, -0.0503,  0.0230, -0.3637, -0.0708,
          0.1103, -0.0739],
        [ 0.0431, -0.0212, -0.1294,  0.1650,  0.0585, -0.0102, -0.3153, -0.1349,
          0.0603, -0.0431]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0949, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1322, grad_fn=<SumBackward0>)